In [18]:
import torch
from torch.utils.data import Dataset
from typing import Tuple
import numpy as np
import requests
import pandas as pd

In [19]:
### LOADING THE MODEL
from torchvision.models import resnet18

In [30]:
model = resnet18(weights=False)
# model.fc = torch.nn.Linear(512, 44)

# ckpt = torch.load("01_MIA.pt", map_location="cpu",weights_only=False)
# model.load_state_dict(ckpt)
# if isinstance(ckpt, dict):
#     print("Top-level keys:", ckpt.keys())

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [22]:
#### DATASETS

class TaskDataset(Dataset):
    def __init__(self, transform=None):

        self.ids = []
        self.imgs = []
        self.labels = []

        self.transform = transform

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]
        if not self.transform is None:
            img = self.transform(img)
        label = self.labels[index]
        return id_, img, label

    def __len__(self):
        return len(self.ids)


class MembershipDataset(TaskDataset):
    def __init__(self, transform=None):
        super().__init__(transform)
        self.membership = []

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int, int]:
        id_, img, label = super().__getitem__(index)
        return id_, img, label, self.membership[index]


data: MembershipDataset = torch.load("./priv_out.pt")

#### EXAMPLE SUBMISSION

df = pd.DataFrame(
    {
        "ids": data.ids,
        "score": np.random.randn(len(data.ids)),
    }
)
df.to_csv("test.csv", index=None)

response = requests.post("http://34.122.51.94:9090/mia", files={"file": open("test.csv", "rb")}, headers={"token": "TOKEN"})
print(response.json())

UnpicklingError: Weights only load failed. In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
Please file an issue with the following so that we can make `weights_only=True` compatible with your use case: WeightsUnpickler error: Unsupported operand 118

Check the documentation of torch.load to learn more about types accepted by default with weights_only https://pytorch.org/docs/stable/generated/torch.load.html.

In [ ]:
import torch
from torch.utils.data import Dataset, DataLoader
from typing import Tuple
import numpy as np
import requests
import pandas as pd
from torchvision import transforms
from torchvision.models import resnet18
import torch.nn.functional as F
from sklearn.linear_model import LogisticRegression
from tqdm import tqdm

#### LOADING THE MODEL

mean = [0.2980, 0.2962, 0.2987]
std = [0.2886, 0.2875, 0.2889]

model = resnet18(pretrained=False)
model.fc = torch.nn.Linear(512, 44)
ckpt = torch.load("./01_MIA.pt", map_location="cpu")
model.load_state_dict(ckpt)

#### DATASETS

class TaskDataset(Dataset):
    def __init__(self, transform=None):
        self.ids = []
        self.imgs = []
        self.labels = []
        self.transform = transform

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int]:
        id_ = self.ids[index]
        img = self.imgs[index]
        if self.transform:
            img = self.transform(img)
        label = self.labels[index]
        return id_, img, label

    def __len__(self):
        return len(self.ids)

class MembershipDataset(TaskDataset):
    def __init__(self, transform=None):
        super().__init__(transform)
        self.membership = []

    def __getitem__(self, index) -> Tuple[int, torch.Tensor, int, int]:
        id_, img, label = super().__getitem__(index)
        return id_, img, label, self.membership[index]

# Load dataset
data: MembershipDataset = torch.load("./priv_out.pt")

# Set up preprocessing and device
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])
data.transform = transform

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

# Enable gradients only for the final layer
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

# DataLoader with small batch size to avoid OOM
loader = DataLoader(data, batch_size=8, shuffle=False)

# Feature extraction
features = []
ids = []

for id_, imgs, labels, _ in tqdm(loader):
    imgs = imgs.to(device)
    labels = labels.to(device)
    imgs.requires_grad = True

    outputs = model(imgs)
    loss = F.cross_entropy(outputs, labels, reduction='none')

    # Compute gradients w.r.t. final layer
    model.zero_grad()
    grads = torch.autograd.grad(loss.sum(), model.fc.parameters(), retain_graph=False)
    flat_grads = torch.cat([g.view(g.size(0), -1) for g in grads], dim=1)
    grad_norms = flat_grads.norm(p=2, dim=1).detach().cpu().numpy()

    # Loss and confidence
    loss_vals = loss.detach().cpu().numpy()
    conf_vals = F.softmax(outputs, dim=1).max(dim=1)[0].detach().cpu().numpy()

    for i in range(len(id_)):
        features.append([grad_norms[i], loss_vals[i], conf_vals[i]])
        ids.append(id_[i])

# Normalize and prepare attack model
features = np.array(features)
features = (features - features.mean(axis=0)) / (features.std(axis=0) + 1e-8)
labels = np.array(data.membership)

# Train logistic regression attack model
attack_model = LogisticRegression()
attack_model.fit(features, labels)

# Predict membership scores
scores = attack_model.predict_proba(features)[:, 1]

# Create submission
df = pd.DataFrame({
    "ids": ids,
    "score": scores,
})
df.to_csv("test.csv", index=None)

# Submit to server
response = requests.post("http://34.122.51.94:9090/mia", files={"file": open("test.csv", "rb")}, headers={"token": "TOKEN"})
print(response.json())
